In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint

In [2]:
#global variabls

directory_path = "/home/hao/AnacondaProjects/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [7]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [9]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)


prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [10]:
print(len(conceptLabelDict))

9955


In [18]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t", 2)
            
            line = splitted[2]
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                conceptMappingDict[i]= int(get_trailing_number(splitted[0]))
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])

In [ ]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f): 
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [randomStr(10),i])

In [19]:
conceptMappingDict={}

train_file = data_path + "ontClassTopology_owl_ncit.txt"

train_corpus = list(read_corpus(train_file))

In [20]:
print(len(train_corpus))
train_corpus[1256:1259]

9955


[TaggedDocument(words=['neoplasm', 'by', 'special', 'category', 'neoplasm', 'by', 'obsolete', 'classification', 'epithelial', 'neoplasm', 'antiquated', 'cancer', 'stage', 'antiquated', 'colorectal', 'cancer', 'by', 'astler', 'coller', 'stage', 'astler', 'coller', 'colorectal', 'carcinoma', 'rectal', 'cancer', 'by', 'astler', 'coller', 'stage', 'astler', 'coller', 'rectal', 'carcinoma', 'astler', 'coller', 'rectal', 'carcinoma', 'astler', 'coller', 'rectal', 'carcinoma'], tags=[1256]),
 TaggedDocument(words=['neoplasm', 'by', 'special', 'category', 'neoplasm', 'by', 'obsolete', 'classification', 'epithelial', 'neoplasm', 'antiquated', 'cancer', 'stage', 'antiquated', 'colorectal', 'cancer', 'by', 'astler', 'coller', 'stage', 'astler', 'coller', 'colorectal', 'carcinoma', 'rectal', 'cancer', 'by', 'astler', 'coller', 'stage', 'astler', 'coller', 'rectal', 'carcinoma', 'astler', 'coller', 'rectal', 'carcinoma', 'astler', 'coller', 'rectal', 'carcinoma'], tags=[1257]),
 TaggedDocument(word

In [22]:
cores = multiprocessing.cpu_count()

print(cores)
models = [
    # PV-DBOW 
    Doc2Vec(dm=0, dbow_words=1, vector_size=200, window=8, min_count=1, epochs=10, workers=cores),
    # PV-DM w/average
    Doc2Vec(dm=1, dm_mean=1, vector_size=200, window=8, min_count=1, epochs =10, workers=cores),
]


8


In [23]:
models[0].build_vocab(train_corpus)
print(str(models[0]))
models[1].reset_from(models[0])
print(str(models[1]))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
Doc2Vec(dm/m,d200,n5,w8,s0.001,t8)


In [25]:
for model in models:
    %%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1min 9s, sys: 756 ms, total: 1min 10s
Wall time: 10.4 s
CPU times: user 36.6 s, sys: 5.11 s, total: 41.7 s
Wall time: 18.1 s


In [30]:
for i, model in enumerate(models):
    temp_path = vector_model_path + "model" + str(i)  # creates a temp file
    model.save(temp_path)

In [27]:
model = Doc2Vec.load("model1") 

pprint(model.docvecs.most_similar([inferred_vector], topn=20))

NameError: name 'inferred_vector' is not defined

In [28]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    pprint(model.docvecs.most_similar([inferred_vector], topn=20))

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
[(730, 0.6486555337905884),
 (9748, 0.6175359487533569),
 (4230, 0.606914758682251),
 (7095, 0.6059582233428955),
 (5881, 0.5948716998100281),
 (1784, 0.5942044258117676),
 (7432, 0.5923898220062256),
 (7411, 0.5885365009307861),
 (6385, 0.5879079103469849),
 (3393, 0.5851361751556396),
 (1677, 0.5828551650047302),
 (2924, 0.5823056697845459),
 (433, 0.581768274307251),
 (8280, 0.5781128406524658),
 (82, 0.5756554007530212),
 (7965, 0.5721471309661865),
 (2417, 0.5712842345237732),
 (541, 0.5674805045127869),
 (7434, 0.5666027069091797),
 (727, 0.5641233921051025)]
Doc2Vec(dm/m,d200,n5,w8,s0.001,t8)
[(7432, 0.5708985924720764),
 (5119, 0.5656453371047974),
 (3975, 0.5338389873504639),
 (3463, 0.5081766843795776),
 (1132, 0.4790615439414978),
 (1875, 0.41995540261268616),
 (5707, 0.40108197927474976),
 (0, 0.39105379581451416),
 (9105, 0.38858065009117126),
 (1674, 0.3786812424659729),
 (234, 0.37764543294906616),
 (5342, 0.37195885181427),
 (8720, 0

In [29]:
for model in models:
    print(str(model))
    inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])
    sims= model.docvecs.most_similar([inferred_vector], topn=20)
    for sim in sims:
        index = str(conceptMappingDict[sim[0]])
        print(conceptLabelDict[index], "score: ", sim[1])
    

Doc2Vec(dbow+w,d200,n5,w8,s0.001,t8)
benign respiratory tract neoplasm score:  0.667535662651062
pulmonary hamartoma score:  0.6600381135940552
atypical adenomatous lung hyperplasia score:  0.6461799144744873
endobronchial hamartoma score:  0.6457448601722717
malignant neoplasm of multiple primary sites score:  0.63677978515625
multiple pulmonary hamartomas score:  0.6347206234931946
congenital peribronchial myofibroblastic tumor score:  0.6337246894836426
malignant neoplasm of lung - multiple primary sites score:  0.632215142250061
benign lung hilum neoplasm score:  0.6287093162536621
axillary neoplasm score:  0.6262967586517334
lung hilum neoplasm score:  0.6115237474441528
giant congenital nevus score:  0.6098372936248779
proliferative dermal lesion in congenital nevus score:  0.6089194416999817
melanoma in congenital melanocytic nevus score:  0.6037662029266357
small congenital melanocytic nevus score:  0.6007400751113892
melanoma arising in giant congenital nevus score:  0.5976440

In [ ]:
#gensim.utils.simple_preprocess??
gensim.models.doc2vec.TaggedDocument??

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=1, iter=5, alpha=0.025, min_alpha=0.025, sample = 1e-3)

gensim.models.doc2vec.Doc2Vec??

model.build_vocab(train_corpus)

len(model.wv.vocab)

%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

# inferred_vector = model.infer_vector(['clinical', 'finding', 'evaluation', 'prevent', 'sampling', 'foot'])
inferred_vector = model.infer_vector(['congenital', 'prolong', 'rupture', 'premature', 'membrane', 'lung'])


sims = model.docvecs.most_similar([inferred_vector], topn=10)
pprint(sims)

for sim in sims:
    print(train_corpus[sim[0]].words, "score: ", sim[1])
    
for sim in sims:
    print(conceptMappingDict[sim[0]], "score: ", sim[1])
  

from tempfile import mkstemp

fs, temp_path = mkstemp("gensim_temp")  # creates a temp file

model.save(temp_path)  # save the model

new_model = gensim.models.Doc2Vec.load(temp_path)

In [ ]:
for model in models:
    print(str(model))
    vec = [model.docvecs["Lady Gaga"] - model["american"] + model["japanese"]]
    pprint([m for m in model.docvecs.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

In [ ]:
print("Total: ",len(train_corpus))


ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    print("Processing: ", doc_id)
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [ ]:
import random, string

def randomStr(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))


In [ ]:
collections.Counter(ranks) 